In [40]:
from pandas import DataFrame, read_csv
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.metrics import accuracy_score
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression 
from sklearn.decomposition import PCA 
from warnings import filterwarnings as fw
fw("ignore")

f1 = pd.read_csv("/home/naksh/Desktop/dataset/train/problem_data.csv")
#f1 = r'train/problem_data.csv'
#df = pd.read_csv(f1)
df=f1
print((df.shape))
f2 = pd.read_csv("/home/naksh/Desktop/dataset/train/train_submissions.csv")
#f2 = r'train/train_submissions.csv'
#train_sub = pd.read_csv(f2)
train_sub=f2

(6544, 4)


In [41]:
df.dropna(subset=['level_type','points','tags'],how='all',inplace=True)
df.shape

(6456, 4)

In [42]:
df.dropna(axis=0,  subset=['level_type'], inplace=True)
df.shape

(6411, 4)

In [43]:
level_type={'A':5,'B':10,'C':15,'D':20,'E':25,'F':30,'G':35,'H':40,'I':45,'J':50,'K':55,'L':60,'M':65,'N':70}
new_points=[level_type.get(i,0) for i in list(df['level_type'])]
df['points']=new_points
df.head()

,problem_id,level_type,points,tags
0,prob_3649,H,40,NaN
1,prob_6191,A,5,NaN
2,prob_2020,F,30,NaN
3,prob_313,A,5,"greedy,implementation"
4,prob_101,A,5,"constructive algorithms,greedy,math"


In [44]:
df.to_csv("/home/naksh/Desktop/dataset/processed.csv", sep=',', encoding='utf-8')

In [45]:
f=r"/home/naksh/Desktop/dataset/processed.csv"
p=pd.read_csv(f)

In [46]:
distinct=train_sub.groupby('problem_id')['user_id'].agg(np.size).reset_index(name='counts')
distinct.head()

,problem_id,counts
0,prob_1,2
1,prob_10,1
2,prob_100,1
3,prob_1000,246
4,prob_1001,10


In [47]:
print(p.shape)
mer=pd.merge(p,distinct,how='left',on='problem_id')
print(mer.shape)
mer=mer.drop(['Unnamed: 0'], axis=1)
mer['counts'].fillna(0,inplace=True)
print(mer.shape)
mer['counts'] = [int(i) for i in mer['counts']]
mer.head(10)

(6411, 5)
(6411, 6)
(6411, 5)


,problem_id,level_type,points,tags,counts
0,prob_3649,H,40,NaN,3
1,prob_6191,A,5,NaN,3
2,prob_2020,F,30,NaN,1
3,prob_313,A,5,"greedy,implementation",17
4,prob_101,A,5,"constructive algorithms,greedy,math",23
5,prob_1479,E,25,NaN,0
6,prob_2940,B,10,NaN,1
7,prob_3059,B,10,brute force,4
8,prob_1958,D,20,NaN,3
9,prob_6154,C,15,NaN,3


In [48]:
# level prediction using no of people SVM

train=mer.iloc[:,:]
test=mer.iloc[:,:]

x_train = train.iloc[:,4]
y_train = train.iloc[:,1]
x_train=[[i] for i in x_train]

x_test = test.iloc[:,4]
y_test = test.iloc[:,1]
x_test=[[i] for i in x_test]

clf = svm.SVC(decision_function_shape='ovo', kernel='linear')
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Accuracy", round(accuracy_score(y_test, y_pred)*100,3))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracy 16.877
Confusion matrix: 
 [[ 91 353   0   0 598   0   0   0   0   0   0   0   0   0]
 [ 62 279   0   0 676   0   0   0   0   0   0   0   0   0]
 [ 41 207   0   0 667   0   0   0   0   0   0   0   0   0]
 [  7 168   0   0 675   0   0   0   0   0   0   0   0   0]
 [  1  82   0   0 712   0   0   0   0   0   0   0   0   0]
 [  0  35   0   0 386   0   0   0   0   0   0   0   0   0]
 [  0  15   0   0 313   0   0   0   0   0   0   0   0   0]
 [  0  15   0   0 257   0   0   0   0   0   0   0   0   0]
 [  0  12   0   0 244   0   0   0   0   0   0   0   0   0]
 [  0  10   0   0 202   0   0   0   0   0   0   0   0   0]
 [  0   8   0   0 146   0   0   0   0   0   0   0   0   0]
 [  0   3   0   0  96   0   0   0   0   0   0   0   0   0]
 [  0   2   0   0  38   0   0   0   0   0   0   0   0   0]
 [  0   2   0   0   8   0   0   0   0   0   0   0   0   0]]
             precision    recall  f1-score   support

          A       0.45      0.09      0.15      1042
          B       0.23      0.

In [49]:
# level prediction using no of people LOGISTIC

x_train = mer[['counts']]
y_train =mer['level_type']

x_test = mer.iloc[:,4]
y_test = mer.iloc[:,1]
x_test=[[i] for i in x_test]

lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
print("Accuracy", round(accuracy_score(y_test, y_pred)*100,3))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred)) 
print(classification_report(y_test, y_pred))


Accuracy 17.345
Confusion matrix: 
 [[380 662   0   0   0   0   0   0   0   0   0   0   0   0]
 [285 732   0   0   0   0   0   0   0   0   0   0   0   0]
 [204 711   0   0   0   0   0   0   0   0   0   0   0   0]
 [123 727   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 59 736   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 22 399   0   0   0   0   0   0   0   0   0   0   0   0]
 [  9 319   0   0   0   0   0   0   0   0   0   0   0   0]
 [  5 267   0   0   0   0   0   0   0   0   0   0   0   0]
 [  4 252   0   0   0   0   0   0   0   0   0   0   0   0]
 [  5 207   0   0   0   0   0   0   0   0   0   0   0   0]
 [  3 151   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  99   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1  39   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  10   0   0   0   0   0   0   0   0   0   0   0   0]]
             precision    recall  f1-score   support

          A       0.35      0.36      0.35      1042
          B       0.14      0.

In [50]:
tags = set()
for i in [i.split(',') for i in list(set(mer['tags']))[1:]]:
    for j in i:
        tags.add(j)
tags = list(tags)
print(len(tags))

36


In [51]:
bow = []
for i in mer['tags']:
    if isinstance(i, float):
        bow.append([0]*len(tags))
        continue
    new = []
    for j in tags:
        new.append(1 if j in i.split(',') else 0)
    bow.append(new)
bow = list(map(list, zip(*bow)))


In [52]:
#adding columns of bag of words to df
for ii, i in enumerate(tags):
    mer[i] = bow[ii]

In [59]:
temp = [ord(i)-65 if not isinstance(i, float) else 3 for i in mer['level_type']]
mer['level_encoded'] = temp

In [60]:
df[:4].head()

,problem_id,level_type,points,tags
0,prob_3649,H,40,NaN
1,prob_6191,A,5,NaN
2,prob_2020,F,30,NaN
3,prob_313,A,5,"greedy,implementation"


In [61]:
mer.head()

,problem_id,level_type,points,tags,counts,geometry,dsu,meet-in-the-middle,ternary search,trees,...,dfs and similar,divide and conquer,chinese remainder theorem,brute force,math,implementation,data structures,combinatorics,graph matchings,level_encoded
0,prob_3649,H,40,NaN,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1,prob_6191,A,5,NaN,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,prob_2020,F,30,NaN,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,prob_313,A,5,"greedy,implementation",17,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,prob_101,A,5,"constructive algorithms,greedy,math",23,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [68]:
# level prediction using no of people and tags LOGISTIC

x_train= mer.iloc[:,4:40]
y_train = mer.iloc[:,1]

x_test = mer.iloc[:,4:40]
y_test = mer.iloc[:,1]

clf = LogisticRegression(multi_class='multinomial',solver ='newton-cg')
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Accuracy", round(accuracy_score(y_test, y_pred)*100,3))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracy 24.068
Confusion matrix: 
 [[854  88  63  16  12   9   0   0   0   0   0   0   0   0]
 [625 170 106  66  43   5   2   0   0   0   0   0   0   0]
 [449 124 144  80 106  12   0   0   0   0   0   0   0   0]
 [399  68  95 115 166   6   1   0   0   0   0   0   0   0]
 [363  31  56  82 251  10   2   0   0   0   0   0   0   0]
 [316   9  19  17  51   9   0   0   0   0   0   0   0   0]
 [290   2   5   5  19   7   0   0   0   0   0   0   0   0]
 [257   2   2   4   3   4   0   0   0   0   0   0   0   0]
 [246   1   2   1   4   2   0   0   0   0   0   0   0   0]
 [205   2   2   1   1   1   0   0   0   0   0   0   0   0]
 [153   0   0   1   0   0   0   0   0   0   0   0   0   0]
 [ 96   0   1   1   1   0   0   0   0   0   0   0   0   0]
 [ 37   3   0   0   0   0   0   0   0   0   0   0   0   0]
 [  9   0   0   0   1   0   0   0   0   0   0   0   0   0]]
             precision    recall  f1-score   support

          A       0.20      0.82      0.32      1042
          B       0.34      0.

In [63]:
# level prediction using no of people BEROULLI
x_train = mer.iloc[:,4]
y_train = mer.iloc[:,1]

x_test = mer.iloc[:,4]
y_test = mer.iloc[:,1]

x_train=[[i] for i in x_train]
x_test=[[i] for i in x_test]

clf = BernoulliNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Accuracy", round(accuracy_score(y_test, y_pred)*100,3))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracy 16.737
Confusion matrix: 
 [[969   0   0   0  73   0   0   0   0   0   0   0   0   0]
 [931   0   0   0  86   0   0   0   0   0   0   0   0   0]
 [835   0   0   0  80   0   0   0   0   0   0   0   0   0]
 [750   0   0   0 100   0   0   0   0   0   0   0   0   0]
 [691   0   0   0 104   0   0   0   0   0   0   0   0   0]
 [354   0   0   0  67   0   0   0   0   0   0   0   0   0]
 [278   0   0   0  50   0   0   0   0   0   0   0   0   0]
 [226   0   0   0  46   0   0   0   0   0   0   0   0   0]
 [198   0   0   0  58   0   0   0   0   0   0   0   0   0]
 [168   0   0   0  44   0   0   0   0   0   0   0   0   0]
 [136   0   0   0  18   0   0   0   0   0   0   0   0   0]
 [ 90   0   0   0   9   0   0   0   0   0   0   0   0   0]
 [ 34   0   0   0   6   0   0   0   0   0   0   0   0   0]
 [  7   0   0   0   3   0   0   0   0   0   0   0   0   0]]
             precision    recall  f1-score   support

          A       0.17      0.93      0.29      1042
          B       0.00      0.

In [65]:
# level prediction using no of people and tags K NEAREST NEIGHBOURS
scaler = StandardScaler()

x_train = mer.iloc[:,4:-1]
y_train=mer.iloc[:,1]

x_test = mer.iloc[:,4:-1]
y_test=mer.iloc[:,1]

classifier = KNeighborsClassifier(n_neighbors=3) #neighbours= 5 accuracy= 30.86 
classifier.fit(x_train, y_train)  
y_pred = classifier.predict(x_test) 
print("Accuracy", round(accuracy_score(y_test, y_pred)*100,3))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracy 31.711
Confusion matrix: 
 [[693  94  44  68   4   0 139   0   0   0   0   0   0   0]
 [312 443  47  69  16   1 129   0   0   0   0   0   0   0]
 [273 121 290  68  20   1 142   0   0   0   0   0   0   0]
 [231 112 101 243  32   6 125   0   0   0   0   0   0   0]
 [172  94  77 100 226   5 121   0   0   0   0   0   0   0]
 [128  39  33  69  21  21 110   0   0   0   0   0   0   0]
 [112  21  25  48   8   1 113   0   0   0   0   0   0   0]
 [ 82  17  18  51   7   1  93   3   0   0   0   0   0   0]
 [ 95  17  16  33   6   2  87   0   0   0   0   0   0   0]
 [ 79  15   8  38   7   0  64   0   0   1   0   0   0   0]
 [ 44   9  17  16   5   3  60   0   0   0   0   0   0   0]
 [ 27   5  17  17   2   2  29   0   0   0   0   0   0   0]
 [ 14   6   1  10   0   0   9   0   0   0   0   0   0   0]
 [  4   1   1   0   0   1   3   0   0   0   0   0   0   0]]
             precision    recall  f1-score   support

          A       0.31      0.67      0.42      1042
          B       0.45      0.

In [67]:
#level prediction using no of people and tags RANDOM FOREST

clf=RandomForestClassifier(n_estimators=50)
x_train = mer.iloc[:,4:-1]
y_train=mer.iloc[:,1]

x_test = mer.iloc[:,4:-1]
y_test=mer.iloc[:,1]

clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)

print("Accuracy", round(accuracy_score(y_test, y_pred)*100,3))
print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

Accuracy 48.417
Confusion matrix: 
 [[816 117  61  12  29   2   0   0   1   4   0   0   0   0]
 [244 650  64  15  37   2   0   0   0   5   0   0   0   0]
 [194  97 558  26  30   2   1   1   1   5   0   0   0   0]
 [188  94  53 455  48   3   1   3   1   4   0   0   0   0]
 [190  72  47  28 443   4   1   1   2   3   1   1   1   1]
 [159  71  51   6  32  95   2   0   3   2   0   0   0   0]
 [169  60  37   3  25   1  29   0   0   4   0   0   0   0]
 [132  59  36   1  17   1   1  19   1   5   0   0   0   0]
 [122  66  36   1  19   1   0   1   9   1   0   0   0   0]
 [ 97  53  28   2  13   0   0   0   0  19   0   0   0   0]
 [ 69  34  31   1   9   0   0   0   2   3   5   0   0   0]
 [ 50  22  12   1   7   0   0   0   2   3   0   2   0   0]
 [ 22   8   2   1   5   0   0   0   0   0   0   0   2   0]
 [  4   3   0   0   0   0   0   0   1   0   0   0   0   2]]
             precision    recall  f1-score   support

          A       0.33      0.78      0.47      1042
          B       0.46      0.